In [1]:
%matplotlib inline
from PIL import Image
from itertools import repeat
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import model_from_json
import keras.callbacks as cb
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [12]:
#Hyperparameter
new_width=64
new_height=128
channel=3
epochCount=12
miniBatchSize=512

In [3]:
def saveModel():
    model_json = model.to_json()
    with open("modelD2.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("modelD2.h5")
    print("Saved model to disk")

In [4]:
class LossHistory(cb.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        batch_loss = logs.get('loss')
        self.losses.append(batch_loss)

In [5]:
#Load pos images from path specified at posList
def loadPosImage(lst):
    result=[]
    for entry in lst:
        im=Image.open(entry, 'r')
        width, height = im.size   # Get dimensions
        left = (width - new_width)/2
        top = (height - new_height)/2
        right = (width + new_width)/2
        bottom = (height + new_height)/2
        im=im.crop((left, top, right, bottom))
        im=np.asarray(im)
        _,_,z=im.shape
        if(z==4):
            im=np.delete(im,3,axis=2)
        #im=im.reshape(new_width*new_height*3)
        result.append(im)
    return result

def loadNegImage(lst):
    scaleRatio=1.5 #tuned with training dataset
    result=[]
    for entry in lst:
        im=Image.open(entry, 'r')
        baseWidth=im.size[0]
        baseHeight=im.size[1]
        im = im.resize((int(baseWidth/scaleRatio),int(baseHeight/scaleRatio)), Image.ANTIALIAS)
        width, height = im.size   # Get dimensions
        width=width-new_width
        height=height-new_height
        #select top left from available range
        curWidth=0
        curHeight=0
        while(curWidth<width):
            curHeight=0
            while(curHeight<height):
                #print(curWidth, curHeight)
                top=curHeight
                left=curWidth
                img=im.crop((left, top, left+new_width, top+new_height))
                img=np.asarray(img)
                _,_,z=img.shape
                if(z==4):
                    img=np.delete(img,3,axis=2)
                #im=im.reshape(new_width*new_height*3)
                result.append(img)
                #move height 128 pixel a time
                curHeight+=128
            curWidth+=64
    return result

In [6]:
def SGDShuffle(inputList, size,sd):
    np.random.seed(sd)
    np.random.shuffle(inputList);
    length=len(inputList)
    batchNum=int(np.ceil(length/size))
    newList=[]
    for i in range(batchNum-2):
        newList.append(inputList[i*size:(i+1)*size])
    newList.append(inputList[(batchNum-1)*size:])
    return np.array(newList)

In [7]:
train_dir = "./INRIAPerson/train_64x128_H96"
neg="neg.lst"
pos='pos.lst'
neg=os.path.join(train_dir, neg)
pos=os.path.join(train_dir, pos)
print("neg images are stored at:",neg)
print("pos images are stored at:",pos)

neg images are stored at: ./INRIAPerson/train_64x128_H96/neg.lst
pos images are stored at: ./INRIAPerson/train_64x128_H96/pos.lst


In [8]:
#Create two lists that contain locations of positive and negative images
posList=[]
negList=[]
with open(neg, 'r') as f:
    for line in f:
        line=line[6:].strip('\n')
        line=os.path.join(train_dir, line)
        negList.append(line)
with open(pos, 'r') as f:
    for line in f:
        line=line[6:].strip('\n')
        line=os.path.join(train_dir, line)
        posList.append(line)
#Repeat each item in negList 3 times, for image reuse
#negList = [x for item in negList for x in repeat(item, 3)]

In [9]:
posImgList=loadPosImage(posList)
negImgList=loadNegImage(negList)
posLength=len(posImgList)
negLength=len(negImgList)
print(posLength, " pos images, ", negLength, "neg images")
#create tag for each image
posTag=[[0,1]]*posLength
negTag=[[1,0]]*negLength
dataList=[]
dataList.extend(posImgList)
dataList.extend(negImgList)
dataList=np.array(dataList)
#dataList=np.append(posImgList, negImgList, axis = 0)
dataTag=[]
dataTag.extend(posTag)
dataTag.extend(negTag)
dataTag=np.array(dataTag)

2416  pos images,  8156 neg images


In [11]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                activation='relu',
                input_shape=(new_height, new_width, 3)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(rate=0.3))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(16, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [13]:
history = LossHistory()
for j in range(epochCount):
    sd=np.random.randint(0,1000)
    mini_images=SGDShuffle(dataList,miniBatchSize,sd)
    mini_labels=SGDShuffle(dataTag,miniBatchSize,sd)
    sd=np.random.randint(0,1000)
    test_images=SGDShuffle(dataList,miniBatchSize,sd)
    test_labels=SGDShuffle(dataTag,miniBatchSize,sd)
    for i in range(len(mini_labels)):
        model.fit(mini_images[i], mini_labels[i], batch_size=len(mini_images[i][0]),
                callbacks=[history],validation_data=(test_images[i], test_labels[i]), verbose=0)
        score = model.evaluate(test_images[i], test_labels[i], batch_size=miniBatchSize
        )
print(score)
plt.plot(history.losses)
plt.title('Loss per batch')
plt.show()

Instructions for updating:
Use tf.cast instead.
512/512 [==============================] - 1s 3ms/step


KeyboardInterrupt: 

In [16]:
saveModel()

Saved model to disk
